# Get data

## Datasets (CSV files)

### Education data

In [ ]:
import pandas as pd

In [ ]:
def summary(df):
    print(df.shape)
    return df.head(10)

In [ ]:
url = 'https://data.cityofchicago.org/api/views/fvrx-esxp/rows.csv?accessType=DOWNLOAD'  # 2015 - 2016

education_df = pd.read_csv(url, usecols=['Zip', 'Student_Attendance_Avg_Pct', 'Suspensions_Per_100_Students_Year_1_Pct'])
education_df.columns = ['zip', 'suspension', 'attendance']
summary(education_df)

In [ ]:
education_df.dropna(subset = ["suspension"], inplace=True)
summary(education_df)
education_df.to_csv('./data/education.csv', index=False)

## Maps (Geojson files)

In [ ]:
import requests

### Zip codes

In [ ]:
url = 'https://data.cityofchicago.org/api/geospatial/gdcf-axmw?method=export&format=GeoJSON'
response = requests.get(url)
open('./data/zip_codes.geojson', 'wb').write(response.content)

### Wards

In [ ]:
url = 'https://data.cityofchicago.org/api/geospatial/sp34-6z76?method=export&format=GeoJSON'
response = requests.get(url)
open('./data/warps.geojson', 'wb').write(response.content)